# Datensammlung von Nachrichtenseiten per Web Scraping - RND

In [22]:
#Installation Bibliotheken

import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import json
import re

In [23]:
#Definition der Parameter für das Web Scraping 

#Suchwörter in Google Search API
search_queries = ["Künstliche Intelligenz", "AI", "Artificial Intelligence", "KI"]

#Lege ein Startdatum für die automatische Datenerfassung fest
last_day_str = "2023-12-28"

#Zeitintervall von x Tagen vor dem Zieldatum
request_days = 3

#Limit für API-Anfragen
api_request_limit = 7

#Pausenzeit des Data Scraping über Beautiful Soup in Sekunden 
scrap_pause = 3

#Bereinigung der Datumsangabe
last_day = datetime.strptime(last_day_str, "%Y-%m-%d")

#Berechnung des letzten Tages
first_day = last_day - timedelta(days=request_days*api_request_limit) - ((timedelta(days=1)*api_request_limit)-timedelta(days=1))

#Ausgabe des Zeitraums
print(f"Time period: {first_day.strftime('%Y-%m-%d')} until {last_day_str} with {api_request_limit} requests")

Time period: 2023-12-01 until 2023-12-28 with 7 requests


In [24]:
#API-Anfrage

#API Schlüssel
api_key = "???"

#DataFrame für die Ergebnisse erstellen
df_data_all_queries = []

#Berechnung Zeitintervall für jede Schleife
loop_interval = timedelta(days=request_days)

#Schleife bis zur Erreichung der Grenze an API-Anfragen
for i in range(api_request_limit):
    #Berechnen des Start- und Enddatums für die aktuelle Schleife
    end_date = last_day - (loop_interval * i) - (timedelta(days=1)*i) if i > 0 else last_day - (loop_interval * i)
    start_date = end_date - timedelta(days=request_days)

    #Bereinigung von Datumsangaben
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    #Schleife über die Suchbegriffe
    for search_query in search_queries:
        #API-Endpunkt und Parameter
        api_endpoint = "https://www.googleapis.com/customsearch/v1"
        api_params = {
            "key": api_key,
            "cx": "???",
            "q": f"{search_query} before:{end_date_str}",
            "sort": "date",
        }

        #API-Aufruf
        response = requests.get(api_endpoint, params=api_params)
        data = response.json()

        #Daten im DataFrame speichern
        if "items" in data:
            for item in data["items"]:
                link = item["link"]
                title = item["title"]

                #Extrahieren von HTML-Inhalten aus der Website
                article_response = requests.get(link)

                #Pause von x Sekunden
                time.sleep(scrap_pause)

                soup = BeautifulSoup(article_response.text, 'html.parser')

                '''Individueller Teil je Quelle'''

                #Extrahieren des Inhalts
                #Suche nach dem Muster '{"type": "text", "text") und Extraktion der Artikeltexte
                pattern = re.compile(r'\{"type":"text","text"(.+?)\}')
                matches = re.findall(pattern, str(soup))

                #Kombinieren der extrahierten Texte
                content = ' '.join(matches)

                #Extrahieren des Datums aus dem HTML-Inhalt
                #Suche nach dem Meta-Tag "datePublished" und Extraktion seines Wertes
                date_published_tag = soup.find('meta', {'name': 'cXenseParse:publishtime'})

                #Prüfen, ob der Meta-Tag gefunden wurde
                if date_published_tag:
                    #Falls gefunden, extrahiere den Wert des Inhaltsattributs
                    date_published_content = date_published_tag.get('content', "")
                else:
                    #Fallback-Lösung: Verwendung des aktuellen Datums und der Uhrzeit im Format 'JJJJ-MM-TTTHH:MM:SSZ'
                    fallback_date = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
                    date_published_content = fallback_date

                #Konvertieren des Wertes in das gewünschte Format
                try:
                    #Versuche, das Datum im vorhandenen Format zu analysieren
                    parsed_date = datetime.strptime(date_published_content, '%Y-%m-%dT%H:%M:%SZ')
                    formatted_date = parsed_date.strftime('%Y-%m-%d')
                except ValueError:
                    #Wenn das Parsing fehlschlägt, setze den Standardwert "1900-01-01"
                    formatted_date = "1900-01-01"

                #Extrahiere den Titel aus der HTML-Datei
                article_title = soup.title.text

                '''Individueller Teil je Quelle'''

                #Prüfen, ob das Datum mit dem Enddatum übereinstimmt
                if datetime.strptime(start_date_str, "%Y-%m-%d") <= datetime.strptime(formatted_date, "%Y-%m-%d") <= datetime.strptime(end_date_str, "%Y-%m-%d"):
                    df_data_all_queries.append({
                        "Search Query": search_query,
                        "Quelle": "RND",
                        "Datum": formatted_date,
                        "Link": link,
                        "Titel": article_title,
                        "Text": content
                    })

#DataFrame für alle Suchbegriffe erstellen
df_all_queries = pd.DataFrame(df_data_all_queries)

#DataFrame anzeigen mit Prüfen, ob der DataFrame leer ist
if df_all_queries.empty:
    print(f"No entries found!")
else:
    print(df_all_queries)

               Search Query Quelle       Datum  \
0                        AI    RND  2023-12-24   
1                        AI    RND  2023-12-21   
2                        KI    RND  2023-12-19   
3    Künstliche Intelligenz    RND  2023-12-14   
4    Künstliche Intelligenz    RND  2023-12-13   
5                        AI    RND  2023-12-14   
6                        KI    RND  2023-12-14   
7                        KI    RND  2023-12-13   
8    Künstliche Intelligenz    RND  2023-12-12   
9    Künstliche Intelligenz    RND  2023-12-11   
10                       AI    RND  2023-12-11   
11  Artificial Intelligence    RND  2023-12-11   
12                       KI    RND  2023-12-12   
13                       KI    RND  2023-12-11   
14                       KI    RND  2023-12-11   
15                       KI    RND  2023-12-09   
16   Künstliche Intelligenz    RND  2023-12-08   
17   Künstliche Intelligenz    RND  2023-12-08   
18   Künstliche Intelligenz    RND  2023-12-08   


In [25]:
#Neue Daten in einem Hauptdatensatz in CSV speichern

#Prüfen, ob die CSV-Datei bereits existiert
try:
    #Versuche, die vorhandene CSV-Datei zu lesen
    existing_df = pd.read_csv("2_Daten_RND_V2.csv")
except FileNotFoundError:
    #Wenn die Datei nicht existiert, erstelle einen leeren DataFrame
    existing_df = pd.DataFrame()

#Anhängen der neuen Daten an den bestehenden DataFrame
df = pd.DataFrame(df_all_queries)
df_to_append = pd.concat([existing_df, df], ignore_index=True)

#Duplikate auf Basis aller Spalten entfernen
df_to_append = df_to_append.drop_duplicates()

#Speichern des kombinierten DataFrame in der CSV-Datei
df_to_append.to_csv("2_Daten_RND_V2.csv", index=False)